In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer.data import import_records
from techminer.features import process_records

import_records(source="../data/tmp/debug-scopus.csv", filetype="scopus", directory="../data/processed/debug")
# process_records(directory="../data/processed/debug")

# wos_df = load_file(filepath="techminer/tests/data/savedrecs.txt", filetype="wos", datastorepath="./datastore")
# dim_df = load_file(filepath="techminer/tests/data/dimensions.csv", filetype="dimensions", datastorepath="./datastore")

In [1]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer.data import coverage 
from techminer.data import describe_records

# coverage(directory_or_records="../data/processed/debug")
describe_records(directory_or_records="../data/processed/debug")

value
Category Item                                               
GENERAL  Documents:                                      479
         Years:                                    2015-2021
         Compound annual growth rate:                50.03 %
         Average citations per document:               13.86
         Average citations per document per year:       1.98
         Total references:                             12322
         Average global references per document:          26
         Source titles:                                    1
         Average documents per publication_name:         479
         Abbreviated Source titles:                        1
AUTHORS  Authors:                                       1678
         Author appearances:                            1817
         Documents per author:                          0.29
         Authors per document:                           3.5
         Single-authored documents:                       52
         Multi-authored documents:                       427
         Co-authors per document:                       3.79
         Collaboration index:                           3.81
         Institutions:                                   589
         Institutions (1st author):                      339
         Countries:                                       61
         Countries (1st author):                          56
KEYWORDS Author Keywords (raw):                         1678
         Index Keywords (raw):                          4035
         Index Keywords (cleaned):                      3937
OTHERS   authors                                        1678
         doi                                             472
         global_references_count                          68
         id                                              472
         pub_year                                          7

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer.query import count_documents_by_term  
from techminer.query import count_documents_by_year
from techminer.query import count_global_citations_by_term
from techminer.query import count_global_citations_by_year
from techminer.query import count_local_citations_by_term
from techminer.query import count_local_citations_by_year
from techminer.query import mean_global_citations_by_year
from techminer.query import mean_local_citations_by_year
from techminer.query import top_documents
# ---
from techminer.visualization import worldmap

# top_documents(directory_or_records="../data/processed/debug")
# count_documents_by_term(directory_or_records="../data/processed/debug", column='authors')
# count_documents_by_year(directory_or_records="../data/processed/debug")
# count_local_citations_by_term(directory_or_records="../data/processed/debug", column='authors')
# count_global_citations_by_term(directory_or_records="../data/processed/debug", column='authors')
# count_local_citations_by_year(directory_or_records="../data/processed/debug")
# count_global_citations_by_year(directory_or_records="../data/processed/debug")
# mean_global_citations_by_year(directory_or_records="../data/processed/debug")
# ---


num_documents_by_country = count_documents_by_term(directory_or_records="../data/processed/debug", column='countries')
worldmap(num_documents_by_country)



In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer.analysis import core_sources
from techminer.analysis import core_authors
from techminer.visualization import lotka

# core_sources(directory_or_records="../data/processed/debug")
x = core_authors(directory_or_records="../data/processed/debug")
lotka(x)

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)


from techminer.visualization import bradford

bradford(directory_or_records="../data/processed/debug")

In [ ]:
import pandas as pd

from techminer.data.records import load_records

sorted(load_records(directory="../data/processed/debug").columns)

In [ ]:
from src.features.apply_keywords_thesaurus import apply_keywords_thesaurus
from src.features.apply_institutions_thesaurus import apply_institutions_thesaurus

# Ok!
apply_keywords_thesaurus(datastoredir="../data/processed/debug")
apply_institutions_thesaurus(datastoredir="../data/processed/debug")

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer.analysis import Bradford

bradford = Bradford(datastoredir="../data/processed/debug")

bradford.plot(max_items=8, cmap='Blues', figsize=(8, 5), fontsize=9)

In [ ]:
bradford.sources_

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from src.analysis import TopDocuments

TopDocuments(datastoredir="../data/processed/debug").detailed_

In [ ]:
from techminer.analysis.top_documents import TopDocuments

TopDocuments(datastorepath="./datastore").print()

In [ ]:
from techminer.analysis.core_authors import CoreAuthors

CoreAuthors(datastorepath="./datastore").table_

In [ ]:
from techminer.analysis.core_authors import CoreAuthors

CoreAuthors(datastorepath="./datastore").plot(figsize=(8, 4))

In [ ]:
from techminer.analysis.core_sources import CoreSources

CoreSources(datastorepath="./datastore").table_

In [ ]:
from techminer.analysis.coverage import Coverage

Coverage(datastorepath="./datastore").table_

In [ ]:
from techminer.analysis.worldmap import WorldMap

WorldMap(datastorepath="./datastore").plot(figsize=(9, 5), colormap='Blues')

In [ ]:
import pandas as pd

pd.read_csv("./datastore/datastore.csv").authors_id